# Pretraining a Model on Twitter Sentiment

In [3]:
import pandas as pd
import numpy as np

## Building a Words List and Word Vector matrix

We need to include Trump's vocab in both the wordlist and the Word2Vec matrix so that we can make predictions on it.

### Wordlist

In [128]:
df = pd.read_csv("./data/random_tweets_sentiment.csv", encoding="latin1", usecols=[1, 2])
df.rename(columns={"Sentiment":"sentiment", "SentimentText":"text"}, inplace=True)
df.head()

FileNotFoundError: File b'./data/random_tweets_sentiment.csv' does not exist

In [129]:
df2 = pd.read_csv("./data/alltweets.csv", encoding="latin1", usecols=[0])
print(df2.shape)
df2.head()

FileNotFoundError: File b'./data/alltweets.csv' does not exist

In [130]:
sentences = df.text.append(df2.text)
sentences.head()

NameError: name 'df' is not defined

In [131]:
sentences.shape

NameError: name 'sentences' is not defined

In [132]:
sentences.isna().sum()

NameError: name 'sentences' is not defined

In [133]:
sentences = sentences.values
len(sentences)

NameError: name 'sentences' is not defined

In [134]:
import re

wordlist = list(set(re.split("( |[^a-zA-Z\d])", "".join(sentences).lower())))
#wordlist = np.array(wordlist)

NameError: name 'sentences' is not defined

In [135]:
# import pickle 

# file = open("./data/tweets_wordlist.pkl", "wb")
# pickle.dump(wordlist, file)

In [136]:
# f = pickle.load(open("./data/tweets_wordlist.pkl", "rb"))

# for x in f:
#     print(x)

In [137]:
len(wordlist)
#wordlist.shape

NameError: name 'wordlist' is not defined

In [138]:
# import pickle 

# file = open("./data/tweets_edited.pkl", "wb")
# pickle.dump(sentences, file)

### WordVector

First we swap words in the words matrix for the indices in the wordlist. We can then use this integer matrix as input to the Word2Vec algorithm.

In [139]:
# Running this on a GPU

# tweets = []
# sizes_sum = 0

# for i in range(len(sentences)):
#     s_lower_split = re.split("( |[^a-zA-Z\d])", "".join(sentences[i]).lower())
#     s_lower_split = list(filter(lambda x : x not in [" ", ""], s_lower_split))
#     s_lower_split = list(map(lambda x : wordlist.index(x), s_lower_split))
#     sizes_sum += len(s_lower_split)
#     tweets.append(s_lower_split)
#     print(i)

In [ ]:
# print(model)

In [ ]:
model.build_vocab

## Building the Word2Vec model

In [1]:
import tensorflow as tf

In [4]:
batch_size = 50
tweets = np.load("data/tweets_integerized.npy")
tweets = [item for sublist in tweets for item in sublist]

couples, labels = tf.keras.preprocessing.sequence.skipgrams(tweets, len(tweets), window_size=2)
bigrams = [x for x, y in zip(couples, labels) if y]
batches = [bigrams[i:i + batch_size] for i in range(0, len(bigrams), batch_size)]

input_inputs = [list(zip(*x)) for x in batches]

In [11]:
import math
num_sampled = 64
graph = tf.Graph()
vocab_size = len(tweets)
embedding_size = 4
with graph.as_default():
    
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
    
    nce_weights = tf.Variable(tf.truncated_normal(
        [vocab_size, embedding_size], stddev=1.0/math.sqrt(embedding_size)))
    
    nce_biases = tf.Variable(tf.zeros([vocab_size]))
    
    
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocab_size))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)
    init = tf.global_variables_initializer()

In [13]:

with tf.Session(graph=graph) as session:
    init.run()
    
    for inputs, labels in input_inputs:
        print("batch")
        inputs = np.array(inputs)
        labels = np.array(labels)
        print(inputs)
        print(labels)
        labels = labels.reshape(batch_size, 1)
        feed_dict = {train_inputs: inputs, train_labels: labels}
        _, cur_loss = session.run([optimizer, loss], feed_dict=feed_dict)

print("yeet done")

batch
[  2200  87897  17395 109281  47985 109281 103665  44295 120680  53416
   7462  17663  60980 120680 121810  92041  76280  76280  53416 120680
  69342  42026  34670  69453  68436   3420 102669  42799  54687 120680
  54283  25215  90012 109281  78892  53416  57633 105781  61867  46834
  23342 120680  44792   9118 107225 123524 120680   1385  46198 120680]
[112820 108065 108065   8043 130649  90012  76280  23342 103665  24950
  66347 130649   2927 120680  28110 110196  34879  90919  76001  69453
  92297  76280 120680  17395 120680 120680 120680 120680  69453  17395
  42026  87897  96378 105781 124006  77979  43846 131745  17395 120680
 109281 131679  72980  69453  76280  80451  14346  76280  39025  93878]


InvalidArgumentError: indices[0] = 2200 is not in [0, 1661)
	 [[Node: embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@GradientDescent/update_Variable/ScatterSub"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable/read, _arg_Placeholder_0_0, embedding_lookup/axis)]]

Caused by op 'embedding_lookup', defined at:
  File "/home/joonho/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/joonho/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/joonho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/joonho/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/joonho/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  Fi